In [1]:
# TensorFlow 라이브러리를 사용한 컨볼루션 신경망 구현 예.
# 이 예제는 손으로 쓴 숫자들을 모와 둔 MNIST 데이터베이스를 사용하고 있습니다.
# (http://yann.lecun.com/exdb/mnist/)

# 작성자: Aymeric Damien
# 수정자: 정 경태
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [2]:
import tensorflow as tf

# MINST 데이터 가져오기
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# 파라메터(Parameters)
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# 네트워크 파라메터(Parameters)
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf 그래프(Graph) 입력
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [4]:
# 간편 레퍼(wrapper) 만들기
def conv2d(x, W, b, strides=1):
    # 바이어스 및 RELU 활성화 Conv2D 래퍼
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D 래퍼
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

# 모델 생성
def conv_net(x, weights, biases, dropout):
    # 입력 이미지 리쉐입(Reshape)
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution 층
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution 층
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected 층
    # conv2 출력을 fully connected 층의 입력에 맞도록 변형
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Dropout 적용
    fc1 = tf.nn.dropout(fc1, dropout)

    # 출력 클래스 예측
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [5]:
# 가중치 및 바이어스 저장층
weights = {
    # 5x5 conv, 1 입력, 32 출력
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 입력, 64 출력
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 입력, 1024 출력
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 입력, 10 출력 (클래스 예측)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# 모델 생성
pred = conv_net(x, weights, biases, keep_prob)

# 손실 및 최적화 정의
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 평가 모델
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 변수 초기화
init = tf.initialize_all_variables()

In [6]:
# 세션 설정
session_conf = tf.ConfigProto()
session_conf.gpu_options.allow_growth = True

# 그래프 실행
with tf.Session(config=session_conf) as sess:
    sess.run(init)
    step = 1
    # 최대 반복횟수에 도달할 때까지 훈련 함
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # 최적화 연산(backprop) 실행
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # 배치의 손실 및 정확도 연산
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("최적화 종료됨!")

    # 256개의 mnist 테스트 이미지 정확도 계산
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 1280, Minibatch Loss= 27678.132812, Training Accuracy= 0.31250
Iter 2560, Minibatch Loss= 13930.714844, Training Accuracy= 0.47656
Iter 3840, Minibatch Loss= 9525.361328, Training Accuracy= 0.58594
Iter 5120, Minibatch Loss= 5593.764160, Training Accuracy= 0.73438
Iter 6400, Minibatch Loss= 3282.314209, Training Accuracy= 0.81250
Iter 7680, Minibatch Loss= 7772.218750, Training Accuracy= 0.74219
Iter 8960, Minibatch Loss= 3809.622314, Training Accuracy= 0.82031
Iter 10240, Minibatch Loss= 2950.701416, Training Accuracy= 0.84375
Iter 11520, Minibatch Loss= 1674.299316, Training Accuracy= 0.89844
Iter 12800, Minibatch Loss= 4662.926758, Training Accuracy= 0.75781
Iter 14080, Minibatch Loss= 1398.327148, Training Accuracy= 0.90625
Iter 15360, Minibatch Loss= 1600.447754, Training Accuracy= 0.91406
Iter 16640, Minibatch Loss= 2466.118408, Training Accuracy= 0.89844
Iter 17920, Minibatch Loss= 1530.633301, Training Accuracy= 0.88281
Iter 19200, Minibatch Loss= 844.833862, Training Accu